In [14]:
import pandas as pd
import numpy as np

In [15]:
c = pd.read_csv("customer_join.csv")
ul = pd.read_csv("use_log.csv")

In [16]:
ul.isnull().sum()

log_id         0
customer_id    0
usedate        0
dtype: int64

In [17]:
c.isnull().sum()

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
mean                0
median              0
amax                0
amin                0
flag1               0
cal_date            0
mem_period          0
dtype: int64

#### 클러스터 사용 K-Means
 - 군집개수 4개
 - mean, median, min, max 변수는 월 이용 회수와 관련된 변수 1~8값을 가짐
 - mem_period 단위 너무 크니 scaling

In [18]:
cc = c[["mean", "median", "amax", "amin", "mem_period"]]
cc.head()

,mean,median,amax,amin,mem_period
0,4.833333,5.0,8,2,47.968131
1,5.083333,5.0,7,3,47.968131
2,4.583333,5.0,6,3,47.968131
3,4.833333,4.5,7,2,47.968131
4,3.916667,4.0,6,1,47.968131


In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [20]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(cc)
x_scaled = pd.DataFrame(x_scaled, columns=["mean", "median", "amax", "amin", "mem_period"])
x_scaled.head()

,mean,median,amax,amin,mem_period
0,-0.281206,-0.133676,0.081177,-0.533620,2.031859
1,-0.140545,-0.133676,-0.379928,-0.021149,2.031859
2,-0.421867,-0.133676,-0.841034,-0.021149,2.031859
3,-0.281206,-0.400393,-0.379928,-0.533620,2.031859
4,-0.796964,-0.667109,-0.841034,-1.046090,2.031859


In [21]:
kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit(x_scaled)

In [22]:
cc["cluster"] = clusters.labels_
cc.head()

C:\Users\user\AppData\Local\Temp\ipykernel_12524\3599466053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc["cluster"] = clusters.labels_


,mean,median,amax,amin,mem_period,cluster
0,4.833333,5.0,8,2,47.968131,1
1,5.083333,5.0,7,3,47.968131,1
2,4.583333,5.0,6,3,47.968131,1
3,4.833333,4.5,7,2,47.968131,1
4,3.916667,4.0,6,1,47.968131,1


In [23]:
cc.rename(columns={"mean" : "월 평균", "median" : "월별 중간값",
                   "amax" : "월 최대값", "amin" : "월 최소값",
                   "mem_period" : "회원기간"}, inplace=True)
cc.head()

C:\Users\user\AppData\Local\Temp\ipykernel_12524\382428698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc.rename(columns={"mean" : "월 평균", "median" : "월별 중간값",


,월 평균,월별 중간값,월 최대값,월 최소값,회원기간,cluster
0,4.833333,5.0,8,2,47.968131,1
1,5.083333,5.0,7,3,47.968131,1
2,4.583333,5.0,6,3,47.968131,1
3,4.833333,4.5,7,2,47.968131,1
4,3.916667,4.0,6,1,47.968131,1


In [25]:
cc.groupby("cluster").count()

,월 평균,월별 중간값,월 최대값,월 최소값,회원기간
cluster,,,,,
0,846,846,846,846,846
1,1251,1251,1251,1251,1251
2,759,759,759,759,759
3,1336,1336,1336,1336,1336


In [30]:
cc.groupby("cluster").mean()

,월 평균,월별 중간값,월 최대값,월 최소값,회원기간
cluster,,,,,
0,8.054608,8.039598,10.009456,6.160757,7.809637
1,4.676091,4.670264,7.231815,2.153477,37.819727
2,3.049270,2.882740,4.745718,1.652174,10.147960
3,5.522518,5.373129,8.743263,2.686377,15.688428


 - 그룹0 : 평균 회원기간이 7일로 가장 짧고 월평균 회수가 8회로 가장 높음
 - 그룹1,3 : 평균 회원기간이 다른구룹보다는 긺. 그룹 1은 그룹3 보다 회원유지 기간은 길지만 월평균 이용회수는 적음
 - 그룹2 : 평균 회원유지기간이 다른 그룹보다 짧음 평균 이용회수도 3회로 가장 낮음

 그룹별 특징을 정의해서 프로모션할 때 사용 가능할수 있다.

#### 차원축소
